In [151]:
import pandas as pd
from IPython.display import display

In [153]:
# Téléchargement des dataframes depuis la base de données des BAAC.
urls_y = {
    2022: {
        "carac" : "https://www.data.gouv.fr/api/1/datasets/r/5fc299c0-4598-4c29-b74c-6a67b0cc27e7",
        "lieux" : "https://www.data.gouv.fr/api/1/datasets/r/a6ef711a-1f03-44cb-921a-0ce8ec975995",
        "usagers" : "https://www.data.gouv.fr/api/1/datasets/r/62c20524-d442-46f5-bfd8-982c59763ec8"
    },
    2023 : {
        "carac" : "https://www.data.gouv.fr/api/1/datasets/r/104dbb32-704f-4e99-a71e-43563cb604f2",
        "lieux" : "https://www.data.gouv.fr/api/1/datasets/r/8bef19bf-a5e4-46b3-b5f9-a145da4686bc",
        "usagers" : "https://www.data.gouv.fr/api/1/datasets/r/68848e2a-28dd-4efc-9d5f-d512f7dbe66f"
    },
    2024 : {
        "carac" : "https://www.data.gouv.fr/api/1/datasets/r/83f0fb0e-e0ef-47fe-93dd-9aaee851674a",
        "lieux" : "https://www.data.gouv.fr/api/1/datasets/r/228b3cda-fdfb-4677-bd54-ab2107028d2d",
        "usagers" : "https://www.data.gouv.fr/api/1/datasets/r/f57b1f58-386d-4048-8f78-2ebe435df868"
    }
}

# Dictionnaire qui contiendra les dataframes en vue de la concaténation

dfs = {}

yrl = [2022,2023,2024]
names = ["carac","lieux","usagers"]

for name in names:
    list_df = []
    for yr in yrl:
        url = urls_y[yr][name]
        df_yr = pd.read_csv(url,sep=";") # Le séparateur utilisé est ";"
        df_yr.insert(1,"year",yr) # Colonne année pour distinguer les accidents entre années en vue des opérations de fusion de dataframes
        list_df.append(df_yr)

    concatenated_df_type = pd.concat(list_df,ignore_index=True) # On fait fi de l'index
    dfs[name] = concatenated_df_type # On associe le dataframe des trois années au type de dataframe

dfs


C:\Users\berna\AppData\Local\Temp\ipykernel_56300\3235580160.py:31: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_yr = pd.read_csv(url,sep=";") # Le séparateur utilisé est ";"
C:\Users\berna\AppData\Local\Temp\ipykernel_56300\3235580160.py:31: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_yr = pd.read_csv(url,sep=";") # Le séparateur utilisé est ";"


{'carac':          Accident_Id  year  jour  mois    an   hrmn  lum dep    com  agg  int  \
 0       2.022000e+11  2022    19    10  2022  16:15    1  26  26198    2    3   
 1       2.022000e+11  2022    20    10  2022  08:34    1  25  25204    2    3   
 2       2.022000e+11  2022    20    10  2022  17:15    1  22  22360    2    6   
 3       2.022000e+11  2022    20    10  2022  18:00    1  16  16102    2    3   
 4       2.022000e+11  2022    19    10  2022  11:45    1  13  13103    1    1   
 ...              ...   ...   ...   ...   ...    ...  ...  ..    ...  ...  ...   
 164521           NaN  2024    10     7  2024  02:05    5  94  94065    2    3   
 164522           NaN  2024    30    11  2024  15:27    1  92  92062    2    1   
 164523           NaN  2024    24    10  2024  20:40    3  29  29068    1    1   
 164524           NaN  2024    30    11  2024  15:40    1  92  92012    2    1   
 164525           NaN  2024    10     7  2024  08:30    1  94  94028    2    1   
 
     

In [154]:
if "an" in dfs["carac"].columns:
    dfs["carac"].drop(columns=["an"],inplace=True) # Colonne an redondante avec colonne year

Observons la longueur de chacun des dataframes :

In [155]:
for n,content in dfs.items(): 
    print(n,len(content))

carac 164526
lieux 196410
usagers 377638


Il est normal que le dataframe "usagers" soit le plus long : en effet, pour chaque accident corporel, on peut compter plusieurs victimes. Il est plus surprenant que "lieux" soit plus long que "carac". 
Observons les doublons :

In [156]:
dfs["lieux"]["Num_Acc"].value_counts()

Num_Acc
202300035508    5
202400050330    5
202300039940    4
202300053680    4
202300036543    4
               ..
202200000026    1
202200000027    1
202200000028    1
202200000029    1
202200000006    1
Name: count, Length: 164526, dtype: int64

In [157]:
dfs["lieux"][dfs["lieux"]["Num_Acc"]==202300035508]

,Num_Acc,year,catr,voie,v1,v2,circ,nbv,vosp,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma
101231,202300035508,2023,4,BOULEVARD DE BEAUSEJOUR,0,NaN,1,0,0,1,0,0,1,NaN,-1,1,0,1,30
101232,202300035508,2023,4,BOULEVARD EMILE AUGIER,0,NaN,3,2,0,1,-1,-1,1,NaN,-1,1,9,1,50
101233,202300035508,2023,4,CHAUSSEE LA MUETTE,0,NaN,3,1,3,1,-1,-1,1,NaN,-1,1,6,1,50
101234,202300035508,2023,4,RUE D ANDIGNE,0,NaN,1,1,0,1,0,0,1,NaN,-1,1,0,1,30
101235,202300035508,2023,4,RUE LARGILLIERE,0,NaN,2,2,0,1,0,0,1,NaN,-1,1,0,1,30


In [158]:
sum(dfs["carac"]["Num_Acc"].value_counts()>1)

0

In [159]:
for name in names:
    print(name,dfs[name].columns)

carac Index(['Accident_Id', 'year', 'jour', 'mois', 'hrmn', 'lum', 'dep', 'com',
       'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long', 'Num_Acc'],
      dtype='object')
lieux Index(['Num_Acc', 'year', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp',
       'prof', 'pr', 'pr1', 'plan', 'lartpc', 'larrout', 'surf', 'infra',
       'situ', 'vma'],
      dtype='object')
usagers Index(['Num_Acc', 'year', 'id_usager', 'id_vehicule', 'num_veh', 'place',
       'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2', 'secu3',
       'locp', 'actp', 'etatp'],
      dtype='object')


"Lieux" indique les différentes voies liées à l'accident lorsqu'il a eu lieu dans une intersection complexe. Il contient par ailleurs différentes informations sur le lieu de l'accident (type de route **catr**, l'état de la surface **surf**, la vitesse maximale autorisée **vma**).
Nous le laissons de côté pour l'instant.

Créons maintenant un premier dataset, qui fusionne les données de 2022,2023, et 2024 pour les dataframes **usagers** et **carac** :

In [160]:
KEY = ["year","Num_Acc"]

df_final = dfs["usagers"].merge(dfs["carac"],on=KEY,how="left") # On garde toutes les colonnes usagers

print(df_final.shape[0]==dfs["usagers"].shape[0])

# df_final et dfs["usagers"] font bien la même longueur.

True


In [161]:
sum(df_final["grav"].isna())

0

La gravité des blessures de chaque victime est renseignée, ce qui nous évite un travail supplémentaire de preprocessing pour la variable d'intérêt **grav**.

>Pour rappel, **grav** peut prendre quatre valeurs : 
>* 1 = Indemne
>* 2 = Tué
>* 3 = Blessé hospitalisé
>* 4 = Blessé léger
    

In [162]:
df_final.columns

Index(['Num_Acc', 'year', 'id_usager', 'id_vehicule', 'num_veh', 'place',
       'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2', 'secu3',
       'locp', 'actp', 'etatp', 'Accident_Id', 'jour', 'mois', 'hrmn', 'lum',
       'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long'],
      dtype='object')